In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#### **1. Kiểm tra tổng quan về tập dữ liệu ban đầu**

In [2]:
raw_pl = pd.read_csv("playlist.csv", sep = '\t')
raw_pl.drop(columns="index", inplace=True)

# tao bản copy để xử lý dữ liệu
new_pl = raw_pl.copy()
new_pl.shape

(2093, 32)

Tập dữ liệu `playlists` ban đầu có  `2093` mẫu và `32` thuộc tính. Tóm tắt về các thuộc tính của tập dữ liệu:

|Thuộc tính|Kiểu dữ liệu|     Ý nghĩa của dữ liệu
|:-----------:|:----------------:|:---------:|
|**artwork_url**|Chuỗi |URL ảnh của playlist|
|**created_at**|Thời gian|Thời gian khởi tạo playlist|  
|**description**|Chuỗi|Mô tả của người đăng tải| 
|**duration**|Số nguyên|Tổng thời gian các track nhạc trong playlist|
|**embeddable_by**|Chuỗi|Đối tượng có quyền nhúng track hoặc playlist này|
|**genre**|Chuỗi|Thể loại nhạc của playlist|
|**id**|Kiểu phân loại|Định danh cho playlist|
|**kind**|Chuỗi|Loại user|
|**label_name**|Chuỗi|Hãng thu âm|
|**last_modified**|Thời gian|Lần chỉnh sửa cuối cùng của tác giả| 
|**license**|Chuỗi|Giấy phép|
|**likes_count**|Số nguyên|Số lượt yêu thích của playlist|
|**managed_by_feeds**|Luận lý|Playlist có được quản lý bởi Feed|
|**permalink**|Chuỗi|Đường dẫn cố định của tài nguyên|
|**permalink_url**|Chuỗi|Đường dẫn đến trang Soundcloud |  
|**public**|Luận lý|Playlist có công khai hay không?|
|**purchase_title**|Chuỗi|     
|**purchase_url**|Chuỗi|       
|**release_date**|Thời gian|Thời gian xuất bản playlist|   
|**reposts_count**|Số nguyên|Số lượt đăng lại playlist|
|**secret_token**|Chuỗi|
|**sharing**|Chuỗi|Chế độ chia sẻ công khai hay riêng tư|
|**tag_list**|Chuỗi|Danh sách các tag của playlist|
|**title**|Chuỗi|Tiêu đề playlist|    
|**uri**|Chuỗi|Đường dẫn API của playlist|    
|**user_id**|Kiểu phân loại|Định danh của người sở hữu playlist|   
|**set_type**|Chuỗi|Định dạng đĩa|
|**is_album**|Luận lý|Playlist có phải album hay không|
|**published_at**|Thời gian|Thời điểm đăng tải playlist|   
|**display_date**|Thời gian|Thời điểm playlist hiển thị|   
|**track_count**|Số nguyên|Số lượng track trong playlist|
|**tracks**|Chuỗi|Danh sách id của các track trong playlist được ngăn cách bởi ","|

In [3]:
new_pl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2093 entries, 0 to 2092
Data columns (total 32 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   artwork_url       1190 non-null   object 
 1   created_at        2093 non-null   object 
 2   description       798 non-null    object 
 3   duration          2093 non-null   int64  
 4   embeddable_by     2093 non-null   object 
 5   genre             1273 non-null   object 
 6   id                2093 non-null   int64  
 7   kind              2093 non-null   object 
 8   label_name        414 non-null    object 
 9   last_modified     2093 non-null   object 
 10  license           2093 non-null   object 
 11  likes_count       2093 non-null   int64  
 12  managed_by_feeds  2093 non-null   bool   
 13  permalink         2093 non-null   object 
 14  permalink_url     2093 non-null   object 
 15  public            2093 non-null   bool   
 16  purchase_title    178 non-null    object 


Theo quan sát đầu tiên thì thuộc tính `secret_token` không có dữ liệu do đó ta sẽ xoá cột này đi

In [4]:
new_pl.drop(columns=["secret_token"], inplace=True)

#### **2. Phân tích các thuộc tính định danh của tập `playlists`**

Dựa vào bảng tóm tắt các thuộc tính của tập dữ liệu, có thể chọn ra những thuộc tính có vai trò định danh cho `playlist` như `id`, `permalink`, `permalink_url` và `uri`

In [5]:
print(f"Số lượng id khác nhau: {new_pl['id'].nunique()}")
print(f"Số lượng permalink khác nhau: {new_pl['permalink'].nunique()}")
print(f"Số lượng permalink_url khác nhau: {new_pl['permalink_url'].nunique()}")
print(f"Số lượng uri khác nhau: {new_pl['uri'].nunique()}")

Số lượng id khác nhau: 2093
Số lượng permalink khác nhau: 2003
Số lượng permalink_url khác nhau: 2093
Số lượng uri khác nhau: 2093


Số lượng các giá trị khác nhau của các thuộc tính này là như nhau, đều đó có nghĩa chúng đóng vai trò là tương tự nhau trong tập dữ liệu, tuy nhiên ở đây chúng ta sẽ chỉ giữ lại hai thuộc tính đó là `id` và `permalink_url` của tập dữ liệu với mục đích định danh cũng như truy cập đến đường dẫn của `playlist`sau này.

In [6]:
new_pl.drop(columns=["permalink", "uri"], inplace=True)

#### **3. Phân tích các thuộc tính có dấu hiệu tương đồng về thông tin**

Theo như bảng tóm tắt các thuộc tính ở trên, có thể thấy các thuộc tính `created_at`, `last_modified`, `release_date`, `published_at` và `display_date` là các thuộc tính chỉ thời gian, mặc dù có ý nghĩa khác nhau nhưng chúng khá tương đồng nhau về mặt thông tin mà chúng mang lại:
- `created_at` là thời điểm mà playlist được khởi tạo.
- `last_modified` là thời điểm mà lần cuối playlist được chỉnh sửa.
- `release_date` là thời điểm mà playlist được xuất bản.
- `published_at` là thời điểm mà playlist chính thức được phát hành.
- `display_date` là thời điểm mà playlist hiển thị trên `Soundcloud`

Nếu xét theo các mốc thời gian như vậy, thì thuộc tính `display_date` sẽ là phù hợp nhất để được giữ lại vì nó là thời điểm mà người sử dụng `Soundcloud` có thể truy cập vào playlist. Do đó, ta sẽ loại bỏ các thuộc tính còn lại.

In [7]:
new_pl.drop(columns=["created_at", "last_modified", "release_date", "published_at"], inplace=True)

#### **4. Phân tích các thuộc tính không đóng góp nhiều vào việc phân tích dữ liệu**

In [8]:
new_pl.isnull().sum()

artwork_url          903
description         1295
duration               0
embeddable_by          0
genre                820
id                     0
kind                   0
label_name          1679
license                0
likes_count            0
managed_by_feeds       0
permalink_url          0
public                 0
purchase_title      1915
purchase_url        1634
reposts_count          0
sharing                0
tag_list            1264
title                  0
user_id                0
set_type            1345
is_album               0
display_date           0
track_count            0
tracks               174
dtype: int64

Sau các bước phân tích ở trên, chúng ta còn lại `25` thuộc tính đối với tập dữ liệu ban đầu, tuy nhiên qua quan sát những thuộc tính còn lại cũng như số lượng giá trị bị thiếu của chúng, ta nhận thấy được có 1 số thuộc tính sẽ không có đóng góp đáng kể cho bước phân tích dữ liệu như sau:
- `artwork_url`
- `label_name`
- `description`
- `kind`
- `embeddable_by`
- `license`
- `managed_by_feeds`
- `public`
- `purchase_url`
- `purchase_title`
- `sharing`
- `tag_list`
- `set_type`
- `title`

Tiến hành loại bỏ các cột trên ra khỏi dữ liệu.

In [9]:
new_pl.drop(columns=["artwork_url", "label_name", "description", "kind", "embeddable_by", "license", "title",
                     "managed_by_feeds", "public", "purchase_url", "purchase_title", "sharing", "tag_list", "set_type"], inplace=True)

**Tổng kết quá trình phân tích và chọn lọc dữ liệu cần thiết:**
1. Loại bỏ cột `index`.
2. Thuộc tính `secret_token` không có dữ liệu do đó sẽ bị xoá đi.
3. Các thuộc tính định danh gồm có `playlist` như:
- `id`
- `permalink`
- `permalink_url`
- `uri`
    - Ta chỉ giữ lại `id` để định danh và `permalink_url` để truy cập đến playlist.
4. Các thuộc tính có sự tương đồng về thông tin mang lại như:
- `created_at`
- `last_modified`
- `release_date`
- `published_at`
- `display_date`
    - Ta chỉ giữ lại `display_date` vì đây là thời điểm mà chính thức mọi người có thể thấy và truy cập vào playlist
5. Các thuộc tính không đóng góp nhiều vào việc phân tích dữ liệu:
- `artwork_url`
- `label_name`
- `description`
- `kind`
- `embeddable_by`
- `license`
- `managed_by_feeds`
- `public`
- `purchase_url`
- `purchase_title`
- `sharing`
- `tag_list`
- `set_type`
- `title`
    - Các cột này sẽ bị loại bỏ.
6. Các thuộc tính còn lại trong tập dữ liệu gồm có:
- `id`
- `permalink_url`
- `genre`
- `duration`
- `likes_count`
- `reposts_count`
- `user_id`
- `is_album`
- `display_date`
- `track_count`
- `tracks`

#### **5. Tiền xử lý dữ liệu**

In [10]:
new_pl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2093 entries, 0 to 2092
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   duration       2093 non-null   int64 
 1   genre          1273 non-null   object
 2   id             2093 non-null   int64 
 3   likes_count    2093 non-null   int64 
 4   permalink_url  2093 non-null   object
 5   reposts_count  2093 non-null   int64 
 6   user_id        2093 non-null   int64 
 7   is_album       2093 non-null   bool  
 8   display_date   2093 non-null   object
 9   track_count    2093 non-null   int64 
 10  tracks         1919 non-null   object
dtypes: bool(1), int64(6), object(4)
memory usage: 165.7+ KB


Theo quan sát thì trong `11` thuộc tính còn lại, có hai thuộc tính bị thiếu dữ liệu là `genre` và `tracks`, ta tiến hành phân tích và xử lý các trường hợp thiếu dữ liệu này

##### **5.1 Phân tích và xử lý thuộc tính `tracks`**

In [11]:
num_miss =  new_pl["tracks"].isnull().sum()
rate_miss = num_miss / len(new_pl) * 100
print(f'Số lượng giá trị bị thiếu của thuộc tính tracks là {num_miss}')
print(f'Phần trăm giá trị bị thiếu của thuộc tính tracks là {round(rate_miss, 2)}%')

Số lượng giá trị bị thiếu của thuộc tính tracks là 174
Phần trăm giá trị bị thiếu của thuộc tính tracks là 8.31%


Có thể thấy số lượng giá trị bị thiếu của cột tracks là rất ít, chỉ chiếm 8.31% toàn bộ. Mặt khác, việc ta phân tích dữ liệu tập dữ liệu `playlist` rỗng không có `track` nào là điều vô nghĩa vì không có gì để nhận xét. Vì lý do đó, chúng ta sẽ tiến hành loại bỏ các `playlist` bị thiếu thuộc tính `tracks` đi.

In [12]:
new_pl.drop(new_pl[pd.isna(new_pl['tracks'])].index, inplace=True)
new_pl = new_pl.reset_index(drop=True)

In [13]:
 new_pl["tracks"].isnull().sum()

0

Số lượng giá trị bị thiếu của cột `tracks` đã là 0, tiếp tục xử lý tiếp thuộc tính `genre`

##### **5.2 Phân tích và xử lý thuộc tính `genre`**

In [14]:
new_pl['genre'].unique()

array([nan, 'Electronic', 'Alternative', 'Country', 'African House Music',
       'Techno Minimal', 'techno', 'Techno', 'SuperSpeedPowerPopRock',
       'Ambient', 'Jazz Fusion, Balearic', 'Soul',
       'Soul, Funk, R&B, Disco, 80s, G.I. DISCO', 'G.I. Disco', 'Muzak',
       'field recordings', 'Experimental', 'Latin', 'Hip Hop',
       'Folk & Singer-Songwriter', 'Brazil', 'DeepHouse', 'Jazz',
       'Freestyle', 'House', 'pop rock ', 'Pop Rock', 'Detroit Techno',
       '"Pop"', 'DANCE & ELECTRONIC', 'Dance | House | Deep',
       'Hardtechno', 'OPERATE', 'Classical',
       '"Dance & Electronic" (Dance/Techno/Disco)', 'Deep House',
       'Downbeat', 'Soundtrack', 'DJ Spinna ', 'HOUSE',
       'Deep\\ House\\ Club', 'soundtrack', 'Rap Hiphop', 'DJ-Sets',
       'Dance', 'innervisions', 'World', 'NTS Radio', 'Hunee',
       'Hunch Music', 'house ', 'Dance | House | Tech House',
       'Experimental Electronic', 'Reggae', 'blog, podcast',
       'Melodic Techno', 'Pop', 'Music From O

- Theo như quan sát sơ bộ thì số lượng các giá trị khác nhau của thuộc tính `genre` là rất nhiều và có rất nhiều những giá trị không phù hợp cũng như rất khó để tiến hành phân tích dữ liệu sau này nếu như chúng ta giữ nguyên như vậy. Thuộc tính `genre` rất quan trọng do đó ta cần phải xử lý cột này và đưa các giá trị về dạng chuẩn.
- Dạng chuẩn ở đây chính là các thể loại âm nhạc (music) và âm thanh (audio) mà hiện nay Soundcloud đang có. File `genre_file.txt` chứa toàn bộ các thể loại của Soundcloud.

In [15]:
with open("genre_file.txt", mode='r') as fin:
    std_genre = fin.read().strip().split('\n')
    
print(f"Số thể loại hiện nay của Soundcloud là {len(std_genre)}")
print(std_genre)

Số thể loại hiện nay của Soundcloud là 41
['Alternative Rock', 'Ambient', 'Classical', 'Country', 'Dance & EDM', 'Dancehall', 'Deep House', 'Disco', 'Drum & Bass', 'Dubstep', 'Electronic', 'Folk & Singer-Songwriter', 'Hip-hop & Rap', 'House', 'Indie', 'Jazz & Blues', 'Latin', 'Metal', 'Piano', 'Pop', 'R&B & Soul', 'Reggae', 'Reggaeton', 'Rock', 'Soundtrack', 'Techno', 'Trance', 'Trap', 'Triphop', 'World', 'Audiobooks', 'Business', 'Comedy', 'Entertainment', 'Learning', 'News & Politics', 'Religion & Spirituality', 'Science', 'Sports', 'Storytelling', 'Technology']


Tiến hành xử lý các giá trị `genre` không chuẩn bằng cách chuyển chúng về NaN (rỗng), để dễ dàng và thuận tiện cho quá trình xử lý ngôn ngữ, chuyển tất cả giá trị về dạng chữ thường cũng như xoá đi hết những khoảng trắng thừa.

In [16]:
new_pl['genre'] = new_pl['genre'].str.strip()
new_pl['genre'] = new_pl['genre'].str.lower()

In [17]:
noise_genre = {
    'superspeedpowerpoprock', 'soul', 'muzak', 'field recordings', 'experimental', 'brazil',
    'freestyle', 'operate', 'downbeat', 'dj spinna', 'dj-sets', 'innervisions', 'nts radio', 'hunee',
    'hunch music', 'blog, podcast', 'music from outer space', 'italectro', 'carl borg', 'fish go deep', 
    'simbad', 'tension', 'boxed melody', 'wooden downtempo', 'bassbient', 'radio','LUV', 'Lets Link',
    'music # adult dance music', 'remix', 'minimal', 'smiley fingers', 'smiley fingers', 'dear deer',
    'beatdown', 'ultrastrtetch', 'u_stretch', 'what about now?', 'tags', 'no life', 'downtown party network',
    'nu disco-funk', 'dana ruh', 'field recording new york outdoor indoor', 'jimpster', 
    'dj', 'drum loops', '2014', 'house & disco & tech shit too','psychobilly / exotica', 'musiques de films', 
    'sifflement', 'synth', 'jungle', 'junkbeats', 'urban sound', 'tech- house', 'raw', 
    'bootleg', 'weird','breakbeat','cozmico', 'agargara', 'salad bowls are a social construct', 
    'genre','serato','good music', 'remixes', 'quality booty music', '90tech-trip','livejam', 'fachwerk', 
    'outernational','gafalow rhythm', 'chill out', 'downtempo', 'breaks', 'sense', 'radio sfx', 'black music', 
    'acid house', 'nouvelle caledonie', 'arno', 'clima', 'raoul k', 'm-a-e','heat','edits', 'free downloads', 
    'eclectic & soulful', 'love rapp', 'guitar looping', 'industrial', 'progressive', 'broken beat',
    'underground HARLOW', 'ir', 'old', 'old spice', 'demou', 'leftfield/idm', 'triptech deep slowtech',
    'chocolatefünk', 'boombap', 'housediscotech', 'mixtape', 'speedcore', 'experimental synth', 'berlin', 'lovers punk',
    'brokenbeat', 'kenny bobien', 'aphex twin', 'van hai', 'intimacy', 'freedom', 'electronic, melancholic', 'beats', 
    'other', 'leftfield electronics', 'estroe', 'idm', 'general records', 'various', 'djmix', 'music',
    'triphip', 'nu jazz | headz | grooves','house | electro | broken beat', 'soul house afro downtempo jazz dubstep',
    'sleepy house and proud of it, bitches', 'deep as fuq muzik','deep as fuq', 'oceanic hour glass', 
    'b-sides', 'album', 'bass','trascendental electronic music','electro', 'hip hobby', 'soundart', 'karizma',
    'good musik', 'blanali', 'byzantinechant', 'spacetechnohousefunk', 'indie elektropop', 'mix', 'waves', 
    'caotico', 'stephenjkroos', 'rickwade', 's3a', 'kez', '4004', 'creativeswingalliance','uglydrums', 'facesrecords', 
    'leftfield & chill out - ambient', 'ray harris', 'podcast', 'ralph lawson', 'spoken word commercial',
    'lounge', 'loop dynamics', 'demo', 'roots', 'score', 'footwork', 'juke | footwork', 'chris karpas',
    'experimental \\ post rock \\ electronica', 'spacerock', 'af', 'downtown party network',
    'chris karpas', 'grant boden', 'phd', 'joyfull family records', 'childrens songs', 'bach',
    'bytebeat', 'eam', 'eam, drone, noise', 'esoteric'}

for index, val in enumerate(new_pl["genre"].tolist()):
    # chỉ xét các dòng có genre <> NaN
    if not pd.isna(val):
        if val in noise_genre:
            new_pl.loc[index, 'genre'] = np.nan

In [18]:
new_pl['genre'].unique()

array([nan, 'electronic', 'alternative', 'country', 'african house music',
       'techno minimal', 'techno', 'ambient', 'jazz fusion, balearic',
       'soul, funk, r&b, disco, 80s, g.i. disco', 'g.i. disco', 'latin',
       'hip hop', 'folk & singer-songwriter', 'deephouse', 'jazz',
       'house', 'pop rock', 'detroit techno', '"pop"',
       'dance & electronic', 'dance | house | deep', 'hardtechno',
       'classical', '"dance & electronic" (dance/techno/disco)',
       'deep house', 'soundtrack', 'deep\\ house\\ club', 'rap hiphop',
       'dance', 'world', 'dance | house | tech house',
       'experimental electronic', 'reggae', 'melodic techno', 'pop',
       'reggae dancehall', 'sci-fi ballads', 'electronic/dance', 'rock',
       'alternative rock', 'electronic pop', 'dance | house',
       'religion & spirituality', 'hip-hop & rap', 'jazz hip-hop',
       'dance | techno', 'soulful house', 'disco', 'nu disco',
       'electronica', 'dub techno', 'dance - soulful house',
     

Tiến hành xử lí dữ liệu bị nhiễu bằng cách đưa chúng về dạng chuẩn các thể loại hiện có trên Soundcloud đã nói ở trên, riêng những ô bị thiếu dữ liệu thì sẽ được thay thế bằng `general` nghĩa là những `playlist` này được xem như là tổng hợp các thể loại nhạc chứ không thuộc thể loại nhất định nào cả.

In [19]:
alt_genre = [
    ['alternative', 'new wawe alternative', 'alternative rock'],
    ['ambient techno', 'ambient electronic', 'ambient'],
    ['contemporary classical', 'modern classical', 'classical'],
    ['deephouse', 'deep\\ house\\ club', 'deep\\ house', 'deep', 'deep heat',
     'deep / techno', 'deep techno', 'deep techhouse', 'deep house'],
    ['dance & electronic', '"dance & electronic" (dance/techno/disco)', 'dance & electronic/techno', 'dance & edm'],
    ['dance | house | deep', 'dance', 'dance | house | tech house', 'dance | house', 'dance | techno',
     'dance - soulful house', 'dance | house | disco', 'dancehall'],
    ['nu disco', 'g.i. disco', 'nu-disco', 'disco rap', 'e-boogie/new disco', 'italo disco', 'disco'],
    ['experimental electronic', 'electronic/dance', 'electronic pop', 'electronca', 'electronca',
     'elettronica', 'electronica', 'electronica/dance', 'electronic music', 'electronic'],
    ['hip hop', 'rap hiphop', 'hip hop/rap', 'hip-hop/rap', 'hiphop', 'hip-hop', 'rap', 
     'underground', 'jazz hip-hop', 'polka rap', 'hip-hop & rap'],
    ['african house music', 'soulful house', 'tech\\ house', 'house electronic', 'tech house', 'raw house',
     'house / techno', 'house'],
    ['indie dance', 'indie dance / nu disco', 'indietronica', 'indie pop', 'indie'],
    ['jazz fusion, balearic', 'jazz', 'jazz funk', 'jazz & blues'],
    ['solo piano', 'piano'],
    ['pop rock', '"pop"', 'pop/electronica', 'pop electronic', 'kraut pop', 'pop'],
    ['soul, funk, r&b, disco, 80s, g.i. disco', 'r&b', "r'n'b", 'r&b & soul'],
    ['reggae dancehall', 'reggae'],
    ['techno minimal', 'detroit techno', 'hardtechno', 'melodic techno', 'dub techno',
     'techno / techhouse', 'tech house', 'tech', 'techno / tech house', 'techno / electro'
     'techno dub', 'techno / electro', 'techno dub', 'techno'],
    ['sci-fi ballads', 'ballad'],
    [np.nan, 'general']
]

for alt in alt_genre:
    new_pl["genre"] = new_pl["genre"].replace(alt[:-1], alt[-1])

In [20]:
new_pl['genre'].unique()

array(['general', 'electronic', 'alternative rock', 'country', 'house',
       'techno', 'ambient', 'jazz & blues', 'r&b & soul', 'disco',
       'latin', 'hip-hop & rap', 'folk & singer-songwriter', 'deep house',
       'pop', 'dance & edm', 'dancehall', 'classical', 'soundtrack',
       'world', 'reggae', 'ballad', 'rock', 'religion & spirituality',
       'drum & bass', 'lo-fi', 'indie', 'metal', 'comedy', 'storytelling',
       'piano'], dtype=object)

Kiểm tra tập dữ liệu sau khi đã xử lý xong cả hai trường bị thiếu dữ liệu là `tracks` và `genre`

In [21]:
new_pl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1919 entries, 0 to 1918
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   duration       1919 non-null   int64 
 1   genre          1919 non-null   object
 2   id             1919 non-null   int64 
 3   likes_count    1919 non-null   int64 
 4   permalink_url  1919 non-null   object
 5   reposts_count  1919 non-null   int64 
 6   user_id        1919 non-null   int64 
 7   is_album       1919 non-null   bool  
 8   display_date   1919 non-null   object
 9   track_count    1919 non-null   int64 
 10  tracks         1919 non-null   object
dtypes: bool(1), int64(6), object(4)
memory usage: 151.9+ KB


Có thể thấy là dữ liệu đã được điền đầy đủ, tuy nhiên kiểu dữ liệu của chúng có vẻ chưa phù hợp!

##### **5.3 Biến đổi thuộc tính**

- Các thuộc tính `id`, `genre`, `permalink_url`, `user_id` và `tracks` cần được đổi về `string`
- Thuộc tính `display_date` chuyển về dạng `datetime` với format `DD-MM-YYYY`

In [22]:
new_pl['id'] = new_pl['id'].astype('string')
new_pl['genre'] = new_pl['genre'].astype('string')
new_pl['permalink_url'] = new_pl['permalink_url'].astype('string')
new_pl['user_id'] = new_pl['user_id'].astype('string')
new_pl['tracks'] = new_pl['tracks'].astype('string')

In [23]:
new_pl['display_date'] = pd.to_datetime(new_pl['display_date']).dt.strftime('%d-%m-%Y')

##### **5.4 Sắp xếp lại thứ tự các thuộc tính**

In [24]:
new_pl = new_pl[["id", "permalink_url", "genre", "duration", "likes_count", "reposts_count", "user_id",
                "is_album", "display_date", "track_count", "tracks"]]

Xem lại tập dữ liệu cũng như thông tin về các trường của tập dữ liệu

In [25]:
new_pl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1919 entries, 0 to 1918
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             1919 non-null   string
 1   permalink_url  1919 non-null   string
 2   genre          1919 non-null   string
 3   duration       1919 non-null   int64 
 4   likes_count    1919 non-null   int64 
 5   reposts_count  1919 non-null   int64 
 6   user_id        1919 non-null   string
 7   is_album       1919 non-null   bool  
 8   display_date   1919 non-null   object
 9   track_count    1919 non-null   int64 
 10  tracks         1919 non-null   string
dtypes: bool(1), int64(4), object(1), string(5)
memory usage: 151.9+ KB


#### **6. Lưu dữ liệu**

In [26]:
new_pl.to_csv("New_Playlists.csv", index=False)